## Module 7 - Network Practice

In this notebook, we will see how to create network visualizations using `networkD3`. 


You may have to **rerun** cells to get the network output for networkD3 examples. 


Here are some references:

- [networkD3 reference](https://cran.r-project.org/web/packages/networkD3/networkD3.pdf)

- [network data sources](http://www-personal.umich.edu/~mejn/netdata/)


## networkD3 Library 

`networkD3` library is an R interface to the **D3 Javascript library**. 


Let's do similar layouts as in the igraph practice notebook for networkD3. **This library produces interactive graphs, you can click on a node and drag it, for example.** 

We need two sets of vertices as input to create a graph from them: source vertices, and target vertices. 

In [ ]:
library(networkD3)
library(igraph)

# Create edge data
src <- c("A", "A", "A", "A",
        "B", "B", "C", "C", "D")

target <- c("B", "C", "D", "J",
            "E", "F", "G", "H", "I")
src
target

In [ ]:
networkData <- data.frame(src, target)
networkData

In [ ]:
# Plot - may need to run twice 
simpleNetwork(networkData, height=500)

**We can also convert the `igraph` graphs to `D3` graphs.**


In [ ]:
# Let's recreate the gSA graph from the igraph practice notebook here: 

adj_matrix <- as.matrix(read.table("/dsa/data/all_datasets/networks/SAcountries.txt",header=FALSE, sep=" "))
SAnames <- read.csv("/dsa/data/all_datasets/networks/SAcountrylist.txt", header=FALSE)
gSA <-  graph_from_adjacency_matrix(adj_matrix, weighted = NULL,  mode = "undirected")
gSA <- simplify(gSA)
gSA <- set_vertex_attr(gSA, "label", value= as.vector(SAnames$V1))

In [ ]:
gSA_d3 <- igraph_to_networkD3(gSA)
gSA_d3

In [ ]:
# Create force directed network plot - you can zoom into this graph with mouse 

forceNetwork(Links = gSA_d3$links, Nodes = gSA_d3$nodes, Source = 'source', Target = 'target', 
             NodeID = 'name', Group = 'name', height=500, zoom=TRUE, fontSize=20)

### IS THIS GRAPH PLANAR ? MOVE THE VERTICES WITH MOUSE TO GET RID OF ALL EDGE CROSSINGS IF YOU CAN.


---




Let's look at a more complex data set. This dataset contains the **co-occurance network of characters in Victor Hugo's novel *Les Misérables*.** 


**A vertex represents a character**, and an **edge between two vertices shows that these two characters appeared in the same chapter** of the the book. The **weight** of each edge indicates how often such a co-appearance occured.

In [ ]:
# This is the Les Miserables data set that comes with the library 
data(MisLinks)
data(MisNodes)

Edges (links) have **weights** that are represented by the `value` column in the data frame as below: 

In [ ]:
head(MisLinks) # EDGE DATA - has weights 

Vertices (nodes) have two attributes: `group` and `size`. 

In [ ]:
head(MisNodes) # VERTEX DATA - has size and grouping 

**Can you identify how many different visual channels represent attributes about the data set below?** 

In [ ]:
# Plot - Move mouse over VERTICES to see character names 

forceNetwork(Links = MisLinks, Nodes = MisNodes,
            Source = "source", Target = "target",
            Value = "value", NodeID = "name",
            Group = "group", opacity = 0.8, height=600, zoom=TRUE, fontSize=30)

**Look at [here](https://www.rdocumentation.org/packages/networkD3/versions/0.4/topics/forceNetwork) for all the options you can use with `forceNetwork()` function.**

### Here are some links that show alternative visualizations for the same data set: 
 - https://bost.ocks.org/mike/miserables/
 - https://studentwork.prattsi.org/infovis/labs/character-networks-visualization-for-les-miserables/
 - https://studentwork.prattsi.org/infovis/visualization/les-miserables-character-network-visualization/

### YOUR TURN: 

**Create an interactive network** for the South America data from above. You'll need to create two data frames similar to the `Mis` example. 

In [ ]:
#< YOUR CODE HERE >

# we need a "links" data frame and a "nodes" data frame. 

# we can use SAnames for the nodes data frame. Since we don't need a grouping or size for the nodes, we can skip those columns. 
SAnames

In [ ]:
# for the "links" data frame, we need to convert the adjacency matrix to a data frame with 
# source and target nodes like in the MisLinks data frame. We don't have weights for SA network
# so we can simply assign 1 for value. 
adj_matrix

In [ ]:
# You can create that data frame in any way you want (even by manually listing all relations)
# Following is a quick way of getting it from the matrix: 

library(reshape2)
# the nodes have to be represented by integers starting from 0!
rownames(adj_matrix) <- seq_len(13)-1
colnames(adj_matrix) <- seq_len(13)-1
adj_matrix

In [ ]:
# we use melt to get a list from the matrix 
adj_list <- melt(adj_matrix)
adj_list

In [ ]:

# Then, we REMOVE all the rows where value is zero (no connection between nodes) 
adj_list <- adj_list[adj_list$value==1,]
adj_list

In [ ]:

# Then, we call the function and pay attention to the column names in the links and nodes 
# (since we don't have grouping, we just assign 1 to all nodes)
forceNetwork(Links = adj_list, Nodes = SAnames,
            Source = "Var1", Target = "Var2",
            Value = "value", NodeID = "V1",
            Group = 1, opacity = 0.8, height=500, zoom=TRUE, fontSize=30)

### We can also create it from scratch like this: 

In [ ]:
SANodes <- data.frame(name=SAnames$V1) # you could also type all country names here
SANodes

In [ ]:
# links
# each node should be represented by a number starting from zero, 
# same order as in names (0=Argentina, etc. )
SALinks = data.frame(source=c(0,0,0,0, 0,1,1,1,1,2,2,2,2,2, 2, 2, 2,3,4,4, 4,5, 6, 7, 7),
                     target=c(1,2,3,8,11,2,3,8,9,4,6,7,8,9,10,11,12,9,5,9,12,9,10,10,12))
SALinks

In [ ]:
forceNetwork(Links = SALinks, Nodes = SANodes,
            Source = "source", Target = "target", NodeID = "name", Group="name",  
             height=500, zoom=TRUE, fontSize=10, opacityNoHover = 0.7)

Now, **find neighbors of Bolivia** using the `neighbors()` function. For that, you'll need an `igraph` graph. 

In [ ]:
#< YOUR CODE HERE >
# We already have the gSA graph with the labels 

# first, find the vertex for Bolivia 
bo <- V(gSA)$label=='Bolivia'
bo
# this is the vertex with the label "Bolivia"
V(gSA)[bo]

In [ ]:
# now, find the neighbors of it 
ne <- neighbors(gSA, V(gSA)[bo])
ne

# finally, print out the the labels of the neighbor vertices 
ne$label


---


**We can also read graphs in GML format.** 

This example contains an undirected social network of frequent associations between 62 dolphins in a community living off Doubtful Sound, New Zealand, as compiled by Lusseau et al. (2003). 

In [ ]:
gml_data <- read.graph("/dsa/data/all_datasets/networks/dolphins.gml", format=c("gml"))
head(gml_data)

In [ ]:
gml_data <- simplify(gml_data)

dol <- igraph_to_networkD3(gml_data)

forceNetwork(Links = dol$links, Nodes = dol$nodes, NodeID = "name", Group = "name", height=600, zoom=TRUE)

**With this data set, we can experiment with community detection functions of `igraph` by clustering vertices like this:** 

In [ ]:
cfg <- cluster_fast_greedy(as.undirected(gml_data))
plot(cfg, gml_data)

In [ ]:
ceb <- cluster_edge_betweenness(gml_data)
plot(ceb, gml_data)

In [ ]:
clp <- cluster_label_prop(gml_data)
plot(clp, gml_data)

### Radial Networks

We can also read network data in JSON format like in the example below: 

In [ ]:
# example of a radial network given in Javascript's JSON format.
library(jsonlite)
URL <- "https://raw.githubusercontent.com/christophergandrud/networkD3/master/JSONdata/flare.json"

## Convert the data to list format
Flare <- fromJSON(URL, simplifyDataFrame = FALSE)

In [ ]:
# JSON format
Flare

In [ ]:
# Use subset of data for a more readable diagram
Flare$children = Flare$children[1:3]

radialNetwork(List = Flare, fontSize = 20, opacity = 1, height=600)

We can **visualize clusters** with a radial network like this: 

In [ ]:
hc <- hclust(dist(USArrests), "ave")
radialNetwork(as.radialNetwork(hc))